In [6]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
# calculating cosin base simillarity

from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine, correlation

from sklearn.metrics import pairwise_distances

from scipy.spatial.distance import cosine, correlation

In [90]:
movies_df = pd.read_csv('/home/madhuri/ds_venv/mr_venv/be_class/streamline/recomendation/movies.csv')
ratings_df = pd.read_csv('/home/madhuri/ds_venv/mr_venv/be_class/streamline/recomendation/ratings.csv')

In [91]:
#movies_df = pd.read_csv('movies.csv')
#ratings_df = pd.read_csv('ratings.csv')

In [92]:
merged_df = pd.merge(movies_df,ratings_df, on='movieId', how='inner')

In [93]:
# Filter the dataset to get the movies the user has already rated
#user_ratings = merged_df[merged_df['userId'] == user_id]
#new_df = merged_df
df = merged_df
df.shape
df.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [66]:
df['genres'] = df['genres'].str.strip()
df_genres = df['genres'].str.split('|', expand=True)


In [59]:
# Filter rows where at least one genre indicator is not None (1)
filtered_df = df_genres[df_genres.iloc[:, 9:].any(axis=1)]

In [60]:
filtered_df.head()
new_column_names = filtered_df.iloc[0]
filtered_df = filtered_df[1:]
filtered_df.columns = new_column_names

In [61]:
# Reset the index
filtered_df = filtered_df.reset_index(drop=True)

In [62]:
filtered_df

91893,Action,Adventure,Comedy,Crime,Drama,Film-Noir,Horror,Mystery,Thriller,Western
0,Action,Adventure,Comedy,Crime,Drama,Film-Noir,Horror,Mystery,Thriller,Western


In [63]:
df_genres.head()

,0,1,2,3,4,5,6,7,8,9
0,Adventure,Animation,Children,Comedy,Fantasy,None,None,None,None,None
1,Adventure,Animation,Children,Comedy,Fantasy,None,None,None,None,None
2,Adventure,Animation,Children,Comedy,Fantasy,None,None,None,None,None
3,Adventure,Animation,Children,Comedy,Fantasy,None,None,None,None,None
4,Adventure,Animation,Children,Comedy,Fantasy,None,None,None,None,None


In [67]:
df_genres.columns = new_column_names

In [68]:
df_genres.head()

91893,Action,Adventure,Comedy,Crime,Drama,Film-Noir,Horror,Mystery,Thriller,Western
0,Adventure,Animation,Children,Comedy,Fantasy,None,None,None,None,None
1,Adventure,Animation,Children,Comedy,Fantasy,None,None,None,None,None
2,Adventure,Animation,Children,Comedy,Fantasy,None,None,None,None,None
3,Adventure,Animation,Children,Comedy,Fantasy,None,None,None,None,None
4,Adventure,Animation,Children,Comedy,Fantasy,None,None,None,None,None


In [69]:
# Concatenate the original DataFrame with the new genre columns
df = pd.concat([df, df_genres], axis=1)

In [71]:
# Drop the original 'genres' column if needed
df = df.drop(['genres','rating','timestamp'], axis=1)

In [74]:
df.sort_values('userId')

,movieId,title,userId,Action,Adventure,Comedy,Crime,Drama,Film-Noir,Horror,Mystery,Thriller,Western
0,1,Toy Story (1995),1,Adventure,Animation,Children,Comedy,Fantasy,None,None,None,None,None
35548,1777,"Wedding Singer, The (1998)",1,Comedy,Romance,None,None,None,None,None,None,None,None
35249,1732,"Big Lebowski, The (1998)",1,Comedy,Crime,None,None,None,None,None,None,None,None
34348,1676,Starship Troopers (1997),1,Action,Sci-Fi,None,None,None,None,None,None,None,None
2379,50,"Usual Suspects, The (1995)",1,Crime,Mystery,Thriller,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
71056,6387,Once a Thief (Zong heng si hai) (1991),610,Action,Comedy,Crime,Thriller,None,None,None,None,None,None
87454,59915,Stuck (2007),610,Horror,Thriller,None,None,None,None,None,None,None,None
97036,111663,Zombeavers (2014),610,Action,Comedy,Horror,None,None,None,None,None,None,None
98181,122886,Star Wars: Episode VII - The Force Awakens (2015),610,Action,Adventure,Fantasy,Sci-Fi,IMAX,None,None,None,None,None


In [73]:
df.sort_values('userId')

,movieId,title,userId,Action,Adventure,Comedy,Crime,Drama,Film-Noir,Horror,Mystery,Thriller,Western
0,1,Toy Story (1995),1,Adventure,Animation,Children,Comedy,Fantasy,None,None,None,None,None
35548,1777,"Wedding Singer, The (1998)",1,Comedy,Romance,None,None,None,None,None,None,None,None
35249,1732,"Big Lebowski, The (1998)",1,Comedy,Crime,None,None,None,None,None,None,None,None
34348,1676,Starship Troopers (1997),1,Action,Sci-Fi,None,None,None,None,None,None,None,None
2379,50,"Usual Suspects, The (1995)",1,Crime,Mystery,Thriller,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
71056,6387,Once a Thief (Zong heng si hai) (1991),610,Action,Comedy,Crime,Thriller,None,None,None,None,None,None
87454,59915,Stuck (2007),610,Horror,Thriller,None,None,None,None,None,None,None,None
97036,111663,Zombeavers (2014),610,Action,Comedy,Horror,None,None,None,None,None,None,None
98181,122886,Star Wars: Episode VII - The Force Awakens (2015),610,Action,Adventure,Fantasy,Sci-Fi,IMAX,None,None,None,None,None


In [118]:
new_df = merged_df
new_df = new_df.drop(['title','rating','timestamp'], axis=1)
new_df =  new_df[new_df['userId'] == 1]
new_df

,movieId,genres,userId
0,1,Adventure|Animation|Children|Comedy|Fantasy,1
325,3,Comedy|Romance,1
433,6,Action|Crime|Thriller,1
2107,47,Mystery|Thriller,1
2379,50,Crime|Mystery|Thriller,1
...,...,...,...
56816,3744,Action|Crime|Thriller,1
57276,3793,Action|Adventure|Sci-Fi,1
57457,3809,Comedy,1
59170,4006,Adventure|Animation|Children|Sci-Fi,1


In [119]:
#new_df['genres'] = new_df['genres'].str.strip()
#new_df["genres"] = new_df["genres"].str.split("|")
#new_df.explode("genres")

In [117]:
#most_common_genre = new_df['genres'].value_counts().idxmax()

In [120]:
genres = new_df['genres'].str.get_dummies(sep='|')

In [121]:
genres

,Action,Adventure,Animation,Children,Comedy,Crime,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,1,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0
325,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0
433,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
2107,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
2379,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56816,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
57276,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
57457,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
59170,0,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0


In [126]:
df = pd.concat([df, genres], axis=1)
df = df.drop(['rating','timestamp'],axis=1)

In [127]:
df[df['userId'] == 1]

,movieId,title,genres,userId,Action,Adventure,Animation,Children,Comedy,Crime,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,0.0,1.0,1.0,1.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
325,3,Grumpier Old Men (1995),Comedy|Romance,1,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
433,6,Heat (1995),Action|Crime|Thriller,1,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2107,47,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2379,50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,1,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56816,3744,Shaft (2000),Action|Crime|Thriller,1,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
57276,3793,X-Men (2000),Action|Adventure|Sci-Fi,1,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
57457,3809,What About Bob? (1991),Comedy,1,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59170,4006,Transformers: The Movie (1986),Adventure|Animation|Children|Sci-Fi,1,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [128]:
movie_genre_matrix = df.drop(['movieId', 'title','genres','userId'], axis=1)

In [137]:
movie_genre_matrix = movie_genre_matrix.fillna(0)
movie_genre_matrix

,Action,Adventure,Animation,Children,Comedy,Crime,Drama,Fantasy,Film-Noir,Horror,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100831,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100832,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100833,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100834,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [138]:
#movie_similarity = cosine_similarity(movie_genre_matrix, movie_genre_matrix)
movie_genre_matrix.columns

Index(['Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime',
       'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery',
       'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western', 'Action',
       'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Drama',
       'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance',
       'Sci-Fi', 'Thriller', 'War', 'Western', 'Action', 'Adventure',
       'Animation', 'Children', 'Comedy', 'Crime', 'Drama', 'Fantasy',
       'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
       'Thriller', 'War', 'Western'],
      dtype='object')

In [141]:
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
# Assuming your movie_genre_matrix is a dense DataFrame
# Convert it to a sparse matrix
movie_genre_matrix_sparse = csr_matrix(movie_genre_matrix.values)

# Calculate cosine similarity using the sparse matrix
movie_similarity = cosine_similarity(movie_genre_matrix_sparse)


MemoryError: Unable to allocate 75.8 GiB for an array with shape (100836, 100836) and data type float64